The data file contains daily demands for one year in four different regional distribution center for three 
different products.  
Consider a periodic review policy with review interval 6 days and lead time 5 days. The cycle service 
level is 95%. The unit holding cost is $0.15 per unit per day. 
For the regional distribution centers, the inbound transportation cost is $0.09 per unit and the outbound 
transportation cost is $0.10 per unit. 
For product 1, calculate the following for each region: 
1) OUL 
2) Average order quantity 
3) Average Cycle stock 
4) Average Safety Stock 
5) Average Inventory 
6) Daily Average Inventory holding cost 
7) Daily Average transportation cost (sum of inbound and outbound) 
8) The sum of Daily Average Inventory holding cost and Daily Average transportation cost 
 
For product 1, suppose that instead of four regional distribution centers, we will have one National 
distribution center. In the national distributional center case, the inbound transportation cost becomes 
$0.05 per unit and the outbound transportation cost becomes $0.24 per unit. Everything else remains 
the same. In the national distributional center case, calculate the following: 
1) OUL 
2) Average order quantity 
3) Average Cycle stock 
4) Average Safety Stock 
5) Average Inventory 
6) Daily Average Inventory holding cost 
7) Daily Average transportation cost (sum of inbound and outbound) 
8) The sum of Daily Average Inventory holding cost and Daily Average transportation cost 
 
Compare the total daily cost for the regional distribution center and the national distribution center. Do 
you recommend National distribution center? 
Repeat the same exercise for products 2 and 3. Do you see any difference for the last comparison 
question? Why do you think that in some cases, the national distribution center is better, but in other 
cases, the regional distribution center is better? 

In [1]:
#imports

import pandas as pd
import numpy as np


# Load the data from the Excel file
file_path = '/home/jovyan/456/456supplychainanalytics/homework3/Homework3_data.xlsx'
data = pd.read_excel(file_path, sheet_name=None)

# Display the sheet names and first few rows of each sheet to understand the structure of the data
sheet_info = {}
for sheet_name, sheet_data in data.items():
    sheet_info[sheet_name] = sheet_data.head()

sheet_info

{'Product1':      Region1    Region2    Region3    Region4
 0  28.968900  28.041047  32.909375  25.387154
 1  32.266234  29.215401  32.369280  23.987414
 2  24.253116  27.947001  37.979393  19.189852
 3  28.441042  34.829736  28.683706  30.893069
 4  35.251088  35.527830  32.666330  25.563244,
 'Product2':      Region1    Region2    Region3    Region4
 0   9.666331   5.640047   8.737732  10.903474
 1  10.728624  10.534761  14.251344  18.674637
 2  12.363718  10.122809   8.643101   9.428583
 3  13.714988  12.596226  10.244061   7.065937
 4   7.270921  14.265560   6.573278  10.883131,
 'Product3':      Region1    Region2    Region3    Region4
 0  11.761233   8.879787  13.251761  13.247398
 1   8.997187  10.454124   8.666646  11.299595
 2   8.305675  10.828282  11.017132   9.192588
 3   9.967582   8.693183   9.398369  13.390449
 4   9.949685  10.679782  14.840647  12.615613}

In [3]:
# Constants
review_interval = 6
lead_time = 5
cycle_service_level = 0.95
holding_cost_per_unit_per_day = 0.15
inbound_transportation_cost_regional = 0.09
outbound_transportation_cost_regional = 0.10
inbound_transportation_cost_national = 0.05
outbound_transportation_cost_national = 0.24

# Calculate z-value for 95% cycle service level
z_value = 1.645  

# Function to calculate the required metrics for a product in all regions
def calculate_metrics(data, review_interval, lead_time, z_value, holding_cost, inbound_cost, outbound_cost):
    metrics = {}
    
    for region in data.columns:
        daily_demands = data[region].values
        
        # Calculate mean and standard deviation of demand over review and lead time
        mean_demand_review_lead_time = np.mean(daily_demands) * (review_interval + lead_time)
        std_demand_review_lead_time = np.std(daily_demands) * np.sqrt(review_interval + lead_time)
        
        # Calculate OUL
        OUL = mean_demand_review_lead_time + z_value * std_demand_review_lead_time
        
        # Calculate Average Order Quantity (AOQ)
        mean_demand_review_interval = np.mean(daily_demands) * review_interval
        AOQ = OUL - mean_demand_review_interval
        
        # Calculate Average Cycle Stock (ACS)
        ACS = AOQ / 2
        
        # Calculate Average Safety Stock (ASS)
        ASS = z_value * std_demand_review_lead_time
        
        # Calculate Average Inventory (AI)
        AI = ACS + ASS
        
        # Calculate Daily Average Inventory Holding Cost (DAIHC)
        DAIHC = AI * holding_cost
        
        # Calculate Daily Average Transportation Cost (DATC)
        DATC = (AOQ * (inbound_cost + outbound_cost)) / review_interval
        
        # Calculate the sum of Daily Average Inventory Holding Cost and Daily Average Transportation Cost (SDC)
        SDC = DAIHC + DATC
        
        # Store the calculated metrics
        metrics[region] = {
            'OUL': OUL,
            'Average Order Quantity': AOQ,
            'Average Cycle Stock': ACS,
            'Average Safety Stock': ASS,
            'Average Inventory': AI,
            'Daily Average Inventory Holding Cost': DAIHC,
            'Daily Average Transportation Cost': DATC,
            'Sum of Daily Costs': SDC
        }
    
    return metrics

# Calculate metrics for product 1 across all regions
product1_data = data['Product1']
product1_metrics_regional = calculate_metrics(product1_data, review_interval, lead_time, z_value, holding_cost_per_unit_per_day, inbound_transportation_cost_regional, outbound_transportation_cost_regional)

print("Product 1 Metrics for Regional Distribution Centers:")
print(pd.DataFrame(product1_metrics_regional).T)

# Combine demands for all regions to represent a National Distribution Center
national_daily_demand = product1_data.sum(axis=1)

# Calculate metrics for the national distribution center
national_product1_metrics = calculate_metrics(
    pd.DataFrame({'National': national_daily_demand}), 
    review_interval, 
    lead_time, 
    z_value, 
    holding_cost_per_unit_per_day, 
    inbound_transportation_cost_national, 
    outbound_transportation_cost_national
)

print("Product 1 Metrics for National Distribution Center:")
print(pd.DataFrame(national_product1_metrics).T)

Product 1 Metrics for Regional Distribution Centers:
                OUL  Average Order Quantity  Average Cycle Stock  \
Region1  355.307335              176.054645            88.027323   
Region2  356.801393              177.799168            88.899584   
Region3  358.608385              177.673376            88.836688   
Region4  353.294888              175.298579            87.649289   

         Average Safety Stock  Average Inventory  \
Region1             26.677404         114.704726   
Region2             28.630647         117.530231   
Region3             26.894202         115.730890   
Region4             26.968321         114.617610   

         Daily Average Inventory Holding Cost  \
Region1                             17.205709   
Region2                             17.629535   
Region3                             17.359633   
Region4                             17.192642   

         Daily Average Transportation Cost  Sum of Daily Costs  
Region1                           